In [1]:
!pip install pandas deep-translator

  Using cached deep_translator-1.11.4-py3-none-any.whl.metadata (30 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
Using cached deep_translator-1.11.4-py3-none-any.whl (42 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [deep-translator] [beautifulsoup4]

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip


In [8]:
# ============================================================================
# ESTRAZIONE CONTROLLI ENS DA FILE EXCEL CCN-STIC-808
# Copia questo codice in una cella di Jupyter Notebook ed esegui
# ============================================================================

import pandas as pd
import re

# CONFIGURAZIONE - Modifica questi path se necessario
FILE_EXCEL = 'CCN-STIC-808-Verificación-del-cumplimiento-medidas_ANEXO (1).xlsx'
OUTPUT_CSV = 'controlli_ens.csv'

def extract_controls(df, sheet_name):
    """Estrae controlli da un foglio Excel"""
    controls = []
    current_control = None
    current_title = None
    questions = []
    
    for idx, row in df.iterrows():
        col1 = str(row[1]).strip() if pd.notna(row[1]) else ''
        col2 = str(row[2]).strip() if pd.notna(row[2]) else ''
        col3 = str(row[3]).strip() if pd.notna(row[3]) else ''
        
        # Trova codice controllo
        if re.match(r'^(Mp\.|Op\.|Org\.)[a-z]{2,3}\.\d+', col1):
            if current_control and (current_title or questions):
                controls.append({
                    'Codice': current_control,
                    'Titolo': current_title,
                    'Domande': ' | '.join(questions),
                    'Num_Domande': len(questions),
                    'Categoria': sheet_name
                })
            current_control = col1
            current_title = col3
            questions = []
        # Raccogli domande
        elif (col1 == '□' or col1 == '☐') and col2 and len(col2) > 15 and '?' in col2:
            questions.append(col2)
    
    # Ultimo controllo
    if current_control and (current_title or questions):
        controls.append({
            'Codice': current_control,
            'Titolo': current_title,
            'Domande': ' | '.join(questions),
            'Num_Domande': len(questions),
            'Categoria': sheet_name
        })
    
    return controls

def assign_priority(code):
    """Assegna priorità HIGH/MEDIUM/LOW"""
    if code.startswith('Org.') or code.startswith('Op.pl') or code.startswith('Op.ac'):
        return 'HIGH'
    elif code.startswith('Mp.if') or code.startswith('Mp.per'):
        return 'HIGH'
    elif code.startswith('Op.'):
        return 'MEDIUM'
    elif code.startswith('Mp.'):
        return 'MEDIUM'
    return 'LOW'

# ESTRAZIONE
print("🔍 Estrazione controlli in corso...\n")

sheets = [
    'ANEXO II -MARCO ORGANIZATIVO',
    'ANEXO II -MARCO OPERACIONAL',
    'ANEXO II-MEDIDAS DE PROTECCIÓN'
]

all_controls = []
for sheet in sheets:
    df = pd.read_excel(FILE_EXCEL, sheet_name=sheet, header=None)
    controls = extract_controls(df, sheet)
    all_controls.extend(controls)
    print(f"✓ {sheet}: {len(controls)} controlli")

# CREA DATAFRAME
df_controls = pd.DataFrame(all_controls)
df_controls['Priorità'] = df_controls['Codice'].apply(assign_priority)

# RIORDINA
df_controls = df_controls[['Codice', 'Priorità', 'Titolo', 'Num_Domande', 'Domande', 'Categoria']]
priority_map = {'HIGH': 0, 'MEDIUM': 1, 'LOW': 2}
df_controls = df_controls.sort_values(['Codice'], key=lambda x: x.map(lambda c: (priority_map.get(df_controls.loc[x.index, 'Priorità'].iloc[0], 3), c)))

# SALVA
df_controls.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig', sep=';')

# STATISTICHE
print(f"\n{'='*60}")
print(f"✅ Totale: {len(df_controls)} controlli")
print(f"   • HIGH:   {sum(df_controls['Priorità'] == 'HIGH')}")
print(f"   • MEDIUM: {sum(df_controls['Priorità'] == 'MEDIUM')}")
print(f"   • LOW:    {sum(df_controls['Priorità'] == 'LOW')}")
print(f"{'='*60}")
print(f"\n💾 File salvato: {OUTPUT_CSV}\n")

# MOSTRA ANTEPRIMA
print("📋 Anteprima (prime 10 righe):\n")
display(df_controls.head(10))

# Il dataframe è disponibile come 'df_controls'

🔍 Estrazione controlli in corso...

✓ ANEXO II -MARCO ORGANIZATIVO: 0 controlli
✓ ANEXO II -MARCO OPERACIONAL: 132 controlli
✓ ANEXO II-MEDIDAS DE PROTECCIÓN: 81 controlli

✅ Totale: 213 controlli
   • HIGH:   75
   • MEDIUM: 138
   • LOW:    0

💾 File salvato: controlli_ens.csv

📋 Anteprima (prime 10 righe):



,Codice,Priorità,Titolo,Num_Domande,Domande,Categoria
164,Mp.com.1,MEDIUM,Perímetro seguro,0,,ANEXO II-MEDIDAS DE PROTECCIÓN
165,Mp.com.1.1,MEDIUM,¿Se dispone de algún sistema que asegure el pe...,4,¿Se dispone de un sistema de protección perime...,ANEXO II-MEDIDAS DE PROTECCIÓN
167,Mp.com.2,MEDIUM,¿Se emplean redes privadas virtuales cuando la...,0,,ANEXO II-MEDIDAS DE PROTECCIÓN
166,Mp.com.2,MEDIUM,Protección de la confidencialidad,0,,ANEXO II-MEDIDAS DE PROTECCIÓN
168,Mp.com.2.r1,MEDIUM,¿Se emplean algoritmos y parámetros autorizado...,0,,ANEXO II-MEDIDAS DE PROTECCIÓN
169,Mp.com.2.r2,MEDIUM,Respecto a las VPN ¿se emplean dispositivos co...,0,,ANEXO II-MEDIDAS DE PROTECCIÓN
170,Mp.com.2.r3,MEDIUM,,2,¿Se emplean dispositivos hardware en el establ...,ANEXO II-MEDIDAS DE PROTECCIÓN
171,Mp.com.2.r4,MEDIUM,¿Se emplean cifradores que cumplan con los req...,0,,ANEXO II-MEDIDAS DE PROTECCIÓN
172,Mp.com.2.r5,MEDIUM,¿Se cifra toda la información transmitida?,0,,ANEXO II-MEDIDAS DE PROTECCIÓN
173,Mp.com.3,MEDIUM,Protección de la integridad y de la autenticidad,0,,ANEXO II-MEDIDAS DE PROTECCIÓN
